# ML Modelling

* The main thing to see in this notebook are the functions whhich will be printing a systematic report for you machine learning algorithms

* If your'e reading it means you are a begginer and came here in search of a project for your internship or placement, So the codes are self-explanatory and it is kind of a well documented code which you can use for companies in analytics domain since it is kind of a analysis based project.

* From here you can have a good start in Data Science and please after repeating this notebook don't forget to upload your notebook in this notebook section so it will boost your confidence in showing your work to the community.

* for hindi speakers - "upvote karna mat bhoolna tabhi to bhai ki notebook chalegi"

* for others - "please Don't forgot to upvote"

# Importing Packages

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,KFold,RandomizedSearchCV
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.preprocessing import LabelEncoder
import time
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
df=pd.read_csv('../input/airbnb-dataset-for-internship-project-showcase/newyork_airbnb.csv')

now we will convert amenities column ie. we will extract every single values and then join them to our dataset

In [ ]:
# We create an array with the features we want to keep.
selected_features = ['name', 'neighbourhood_cleansed', 'room_type', 'guests_included', 'minimum_nights',
                     'number_of_reviews', 'review_scores_rating', 'amenities', 'property_type',
                     'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'price']
selected_df= df.copy()[selected_features]
selected_df.rename(columns = {'neighbourhood_cleansed':'neighbourhood'}, inplace = True)

In [ ]:
selected_df['price']=selected_df['price'].apply(lambda x: x.replace('$',''))
selected_df['price']=selected_df['price'].apply(lambda x: float(x.replace(',','')))

In [ ]:

##%%time
regex = r"{([^}]*)}"
regex2 = r"translation.\w\D+.."
listings_cp = selected_df.copy()
listings_cp['amenities'] = selected_df['amenities'].map(lambda amns: re.search(regex, amns).group(1))
listings_cp['amenities'] = selected_df['amenities'].map(lambda amns: re.sub(regex2, '', amns))
listings_cp['amenities'] = selected_df['amenities'].map(lambda amns: amns.replace("\"", ""))

In [ ]:

# The code for adding the amenities colums is currently commented for practicality
#amenity_ohe = listings_cp.amenities.str.get_dummies(sep = ",")
#amenities_cols = amenity_ohe.columns.values
# dataset = pd.concat([listings_cp, amenity_ohe], axis=1)
dataset = selected_df.query('price <= 500')
dataset = dataset.drop('amenities', axis=1)
dataset = dataset.drop('name', axis=1)

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.dropna(inplace=True)


# Handling Null Values  

Since there was many empty columns in our dataset so we have to treat them before fitting to the model

In [ ]:
# The numeric features that present null values are: review_scores_rating, bathrooms, bedrooms
# and beds from previous notebook https://github.com/Summi-bhai/Airbnb_dataset/blob/master/Analysis_Part_1.ipynb

dataset['review_scores_rating'] = dataset['review_scores_rating'].fillna(dataset['review_scores_rating'].median())
dataset['bathrooms'] = dataset['bathrooms'].fillna(dataset['bathrooms'].median())
dataset['bedrooms'] = dataset['bedrooms'].fillna(dataset['bedrooms'].median())
dataset['beds'] = dataset['beds'].fillna(dataset['beds'].median())

# Converting categorical features to float


In [ ]:
dataset.dtypes

In [ ]:

# it was showing error with bed_type that's why we will be handling it by our hands
cat_columns = ['neighbourhood', 'room_type', 'property_type','bed_type']
encoder=LabelEncoder()
for col in cat_columns:
    dataset[col]=encoder.fit_transform(dataset[col])

# converting features to float
dataset = dataset.astype(float)
dataset.dtypes

# Splitting the data 


In [ ]:
y=dataset['price']
dataset.drop('price',axis=1,inplace=True)

X=dataset

# now we have training,validation and test data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2)



Now we are going to define .fit() and cross-validation methods then we will fit different different models of regression from sklearn library and then finally use randomized search cv for better hyper-parameter choice 

In [ ]:

def fit_model(model,X_train, y_train, X_val, y_val, cross_val=False, cv_folds=5):
    
    model.fit(X_train,y_train)
    #Predict values:
    training_predictions = model.predict(X_train)
    validation_predictions = model.predict(X_val)
    model_report(X_train,y_train, X_val,y_val, training_predictions, validation_predictions)

    if cross_val:
        evaluate_cross_validation(model, X_train, y_train, cv_folds)
        
def model_report(X_train,y_train, X_val,y_val, training_predictions, validation_predictions):
    #Print model report:
    print("\nModel Report")
    print("Training")
    print("Mean Absolute Error : {}".format(mean_absolute_error(y_train, training_predictions)))
    print("Root Mean Squared Error : {}".format(np.sqrt(mean_squared_error(y_train, training_predictions))))
    print("R2 Score: {}".format(r2_score(y_train, training_predictions)))
    print("\n")
    print("Validation")
    print("Mean Absolute Error : {}".format(mean_absolute_error(y_val, validation_predictions)))
    print("Root Mean Squared Error : {}".format(np.sqrt(mean_squared_error(y_val, validation_predictions))))
    print("R2 Score: {}".format(r2_score(y_val, validation_predictions)))
    
def evaluate_cross_validation(model, X_train, y_train, K):
    
    cv = KFold(n_splits=K, shuffle=True, random_state=2)
    scores = cross_val_score(model, X_train, y_train, cv=cv)
    print()
    print(scores)
    print("Mean score: {} max is {} and min {}".format(scores.mean(), max(scores), min(scores)))   
    
def random_search(model, param_grid, X_train, y_train, X_val, y_val, cv):
    
    random_search = RandomizedSearchCV(model, param_grid, scoring="neg_mean_squared_error", n_jobs=2, cv=cv)
    random_result = random_search.fit(X_train, y_train)

    # summarize results
    print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))
    means = random_result.cv_results_['mean_test_score']
    stds = random_result.cv_results_['std_test_score']
    params = random_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    validation_predictions = random_result.predict(X_val)

    print("Validation")
    print("Mean Absolute Error : {}".format(mean_absolute_error(y_val, validation_predictions)))
    print("Root Mean Squared Error : {}".format(np.sqrt(mean_squared_error(y_val, validation_predictions))))
    print("R2 Score: {}".format(r2_score(y_val, validation_predictions)))


# Model selection and training

 For now we will fit are dataset on Decision tree and Random forest.

In [ ]:
model=DecisionTreeRegressor(random_state=8)
fit_model(model,X_train,y_train,X_val,y_val,cross_val=True)

In [ ]:
model=RandomForestRegressor()
fit_model(model,X_train,y_train,X_val,y_val,cross_val=True)

# Hyperparameter Tuning

Now we had dry runned our model over default parameters but however unfortunately our model overfitted the data so now we will tune our parameters with the help of random search cv we will only using random search for decision tree.

In [ ]:
# Tunning Parameters
max_depth = [5, 10, 15, 20]
max_features = ['log2', 'sqrt', 'auto']
min_samples_leaf = [10, 50, 70, 100]
param_grid = dict(max_depth=max_depth, max_features=max_features, 
                  min_samples_leaf=min_samples_leaf)
kfold = KFold(n_splits=5, shuffle=True, random_state=2)

random_search(DecisionTreeRegressor(random_state=8), param_grid, X_train, y_train, X_val, y_val, kfold)

In [ ]:
model = DecisionTreeRegressor(max_depth=20, min_samples_leaf=50, random_state=8)
fit_model(model,X_train,y_train,X_val,y_val,cross_val=True)

#  Conclusion

We have a model which have maximum accuracy of 57% in and min accuracy of 56% terms of R2_score.

